In [ ]:
# import all necessary packages
import pandas as pd
import os, sys, csv
import glob, errno
import numpy as np
import cv2
import copy
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K

import tensorflow as tf

In [ ]:
# import "trainingset_yolo_1" to generate training set of yolo
import trainingset_yolo_1

In [ ]:
# creating training set

# parameter used in create_train_set function of trainingset_yolo_1

# path                   :          path of file trainingset_yolo_1.py or working directory
# file                   :          csv file having info regarding rectangles and thier cordinate in an image
# image_folder           :          path of data images
# input_image_size       :          image size that will be input to yolo architecture 
# no_of_grid             :          Number of grid that a image can have.
# no_of_class            :          Number of class that has to be identified
# no_of_anchor_box       :          Number of maximum objects that can be detected by  a single grid

# returned variables 

# train_x                :          contain all images of size input_image_size: size of train_x is: 
#                                   (# of instances, image_height, image_width, # of channel) : (69, 512, 512, 3)
                
# train_y         :     contain label for each image having siz of :
#                       (# of instances, grid_ row, grid_column, # of anchor_box, label_of_each_anchor_box) : (69, 8, 8, 2, 10)

#                label_of_each_anchor_box contain : [object mid_point_x relative to grid number,
#                                                    object mid_point_Y relative to grid number,
#                                                    object width i.e.: grid crossed on X-axis,
#                                                    object height i.e.: grid crossed on Y-axis,
#                                                    object presence in anchor box i.e.: anchor_confidence,
#                                                    class of object in one_hot_encoding ]

# true_boxes           :      contained all boxes present in image: an image maximum can identify 50 object of configuration [x, y, w, h] per image
#                             having a shape of (69, 1, 1, 1, 50, 4)

# anchor_boxes         :      used to generate anchor Box using K_means algorithm : size (no_of_anchor_box, anchor_box_dimension) 
#                             here anchor_box_dimension is 2 (width of anchor_box, height of anchor_box )    
# original_image_size  :      list of touple of original image size of training images 
#                             ...will be helpful in resizing image after prediction

train_x, train_y, true_boxes, anchor_boxes, original_image_size = trainingset_yolo_1.create_train_set(path = r'/home/bitnami/vikrant/deep_version/extraction_code', file = 'train_set.csv', image_folder = r'/home/bitnami/vikrant/deep_version/went_into_csv', input_image_size = (512, 512), no_of_grid = (8, 8), no_of_class = 5, no_of_anchor_box = 2  )
print("train_x.shape = ",train_x.shape)
print( "train_y.shape = ", train_y.shape)
print( "true_boxes.shape = ", true_boxes.shape)
print( "anchor_boxes.shape = ", anchor_boxes.shape)
print("original_image_size", len(original_image_size))

In [ ]:
#print(original_image_size)

In [ ]:
# writing all training image to train_image folder of size :computation size i.e: input_image_size 

os.chdir(r'/home/bitnami/vikrant/deep_version/train_image')

for index_img in range(train_x.shape[0]):
    cv2.imwrite("train_image"+str(index_img)+".jpg", train_x[index_img]*255) # to normalize to [0 - 255]
    

In [ ]:
# splitting training and testing 

trainingset_last_index = 64
image_channel_used = 1
Train_x = train_x[0:trainingset_last_index,:,:,0:image_channel_used] 
True_boxes = true_boxes[0:trainingset_last_index,:,:,:,:,:]
Train_y = train_y[0:trainingset_last_index,:,:,:]

print(Train_x.shape, Train_y.shape, True_boxes.shape)

In [ ]:
# total no of class possible 

LABELS = ["heading", "paragraph", "table", "list", "image"]


In [ ]:

IMAGE_H, IMAGE_W = 512, 512                              # IMAGE_H: image_height in architecture
#                                                        # IMAGE_W: image_width in architecture  
 
GRID_H,  GRID_W  = 8 , 8                                 # GRID_H: no of grid row
                                                         # GRID_W: no of grid in a single row

BOX              = 2                                     # number of anchor box

CLASS            = len(LABELS)                           # number of classes 

CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')    

OBJ_THRESHOLD    = 0.49           #.3                     #  object threshold to remove lower confidence objects

NMS_THRESHOLD    = 0.45              #.3                 # non max suppression threshold              

ANCHORS          = [1.90603223, 0.14906887, 5.53059165, 0.74886134] 
                                                        # generated anchor box by training set # fixed it at once

NO_OBJECT_SCALE  = 1.0                           # used in custom loss to give less weight to grid where object is not present

OBJECT_SCALE     = 5.0                           # used in custom loss to give more weight to grid where object is  present

COORD_SCALE      = 1.0                           # used in custom loss to scale object co-ordinates 

CLASS_SCALE      = 1.0                          # used in custom loss to scale class label

BATCH_SIZE       = 8              # batch size in training 

WARM_UP_BATCHES  = 0

TRUE_BOX_BUFFER  = 50              # maximum number of object image can have

In [ ]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)


In [ ]:
# Architecture of YOLO

input_image = Input(shape=(IMAGE_H, IMAGE_W, 1)) # input tensor of architecture
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4)) # true_box input tensor

# # Layer 1

x = Conv2D(64, (3,3), strides=(2,2), padding='same', name='conv_1', use_bias=False)(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(2, 2))(x)
print(x.shape)

# Layer 2
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x)
x = BatchNormalization(name='norm_2')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(2, 2))(x)
print(x.shape)

# Layer 2
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x)
x = BatchNormalization(name='norm_8')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
print(x.shape)
# Layer 2
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x)
x = BatchNormalization(name='norm_9')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
print(x.shape)
# Layer 2
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x)
x = BatchNormalization(name='norm_10')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
print(x.shape)

# Layer 3
x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x)
x = BatchNormalization(name='norm_3')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(2, 2))(x)
print(x.shape)

# Layer 4
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x)
x = BatchNormalization(name='norm_4')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
print(x.shape)

# Layer 5
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x)
x = BatchNormalization(name='norm_5')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
print(x.shape)


# Layer 6
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x) #on 1024 it is strides=(2,2)
x = BatchNormalization(name='norm_6')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(2, 2))(x)
print(x.shape)

# Layer 7
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x)
x = BatchNormalization(name='norm_7')(x)
x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling2D(pool_size=(2, 2))(x)
print(x.shape)

# layer 8
x = Dense(BOX * (4 + 1 + CLASS),activation='relu', name='dense_1')(x)
print(x.shape)

#x = Conv2D(BOX * (4 + 1 + CLASS), (1,1), strides=(1,1), padding='same', name='conv_6')(x)
output = Reshape((GRID_H, GRID_W, BOX, 4 + 1 + CLASS))(x)

# small hack to allow true_boxes to be registered when Keras build the model 
# for more information: https://github.com/fchollet/keras/issues/2790
output = Lambda(lambda args: args[0])([output, true_boxes])
print(output.shape)

model = Model([input_image, true_boxes], output)

In [ ]:
# print model summary

model.summary()

In [ ]:
# defining custom loss of YOLO

# y_true         :    actual training label
# y_pred         :    predicted training label
# mask_shape     :    used to create a mask_shape in order to compute different mask: shape = shape=(4,)
# cell_x         :    x value of grid_base  in any grid ...will hold grid number in x dimesion: shape = (1, 8, 8, 1, 1)
# cell_y         :    y value of grid_base in any grid ...will hold grid number in y dimesion: shape = (1, 8, 8, 1, 1)
# cell_grid      :    (x, y) value of grid_base in all grid in all anchor box in all batch:  shape = (16, 8, 8, 2, 2)
# coord_mask     :    having a shape of mask_shape used to mask coords 
# conf_mask      :    having a shape of mask_shape used to mask confidence
# class_mask     :    having a shape of mask_shape used to mask class 
# seen           :    used to define average recall 
# total_recall   :    used to define total recall so far

def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    print("mask_shape = ",mask_shape)
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))
    print("cell_x.shape = ",cell_x.shape)
    print("cell_y.shape = ",cell_y.shape)
    
    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 2, 1])
    print("cell_grid.shape =",cell_grid.shape)
    
    coord_mask = tf.zeros(mask_shape)
    print("coord_mask.shape =",coord_mask)
    
    conf_mask  = tf.zeros(mask_shape)
    print("conf_mask.shape =",conf_mask)

    class_mask = tf.zeros(mask_shape)
    print("class_mask.shape =",class_mask)
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y   
    
    # pred_box_xy:predicted value of (x, y) object mid cordinate relative to grid no. shape: (16, 8, 8, 2, 2)
    # pred_box_wh:predicted value of (w, h) :no of grid crossed in both x, y direction  shape: (16, 8, 8, 2, 2)
    # pred_box_conf: predicted confidence, shape: (16, 8, 8, 2, 1)
    # pred_box_class: predicted class ,  shape: (16, 8, 8, 2, 5)
    
    
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    print("cell_grid = ",cell_grid)
    print("pred_box_xy = ",pred_box_xy)
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    
    ### adjust class probabilities
    pred_box_class = y_pred[..., 5:]
    print("pred_box_class.shape = ",pred_box_class.shape)
    """
    Adjust ground truth
    """
    
    # true_box_xy : true value of object mid points(x, y) relative to grid  shape: (16, 8, 8, 2, 2)
    # true_box_wh : true value of (w, h) :no of grid crossed in both x, y direction  shape: (16, 8, 8, 2, 2) 
    
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
    
    
    ### adjust confidence , trying to find IOU bw true and predicted values 
    
    
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    print("iou_scores.shape = ",iou_scores.shape) # shape :(16, 8, 8, 2)
    
    true_box_conf = iou_scores * y_true[..., 4]    # finding true confidence of boxes
    print("true_box_conf.shape = ",true_box_conf.shape)
    
    
    ### adjust class probabilities
    true_box_class = tf.argmax(y_true[..., 5:], -1) # returns the most likely class 
    print("true_box_class.shape = ",true_box_class.shape)
    """
    Determine the masks
    """
    
    
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    print("coord_mask.shape = ",coord_mask.shape) # shape: (?, ?, ?, ?, 1)
    
    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    
    true_xy = true_boxes[..., 0:2]
    true_wh = true_boxes[..., 2:4]
    print("true_xy.shape = ",true_xy.shape)
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half
   # print("pred_box_xy.shape = ",pred_box_xy.shape)
    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)
    print("pred_xy.shape = ",pred_xy.shape)
    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    
    print("pred_mins.shape = ",pred_mins.shape)
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    print("intersect_mins.shape = ", intersect_mins.shape)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    print("iou_scores.shape = ",iou_scores.shape)
    
    best_ious = tf.reduce_max(iou_scores, axis=4) # best iou for each anchor with any of 50 true boxes
    print("best_ious.shape = ",best_ious.shape) 
    
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * NO_OBJECT_SCALE # give less weight to grids where objects are not likely to have
    
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE
    
    ### class mask: simply the position of the ground truth boxes (the predictors)
    class_mask = y_true[..., 4] * tf.gather(CLASS_WEIGHTS, true_box_class) * CLASS_SCALE       
    
    """
    Warm-up training
    """
    # first few  epochs are here to help the network get familiar with the dataset before the training.
    
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])
    
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(tf.sqrt(true_box_wh)-tf.sqrt(pred_box_wh))     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)
    
    loss = loss_xy + loss_wh + loss_conf + loss_class
    
    nb_true_box = tf.reduce_sum(y_true[..., 4]) #true box confidence sum
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3)) #predicted confidence sum

    """
    Debugging code
    """    
    current_recall = nb_pred_box/(nb_true_box + 1e-6)
    total_recall = tf.assign_add(total_recall, current_recall) 

    loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=10)
    loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=10)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=10)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=10)
    loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=10)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=10)
    loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=10)
    loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=10)
    
    return loss

In [ ]:
# capture all variable used in yolo
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : 50,
}

In [ ]:
optimizer = Adam(lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss=custom_loss, optimizer=optimizer)

In [ ]:
model.fit([Train_x,True_boxes], Train_y, epochs=80, batch_size = 8,validation_split=0.125, verbose=1)

In [ ]:
#SAVING MODELS INTO  /home/bitnami/vikrant/deep_version/saved_model

os.chdir(r'/home/bitnami/vikrant/deep_version/saved_model')
# serialize model to JSON
model_json = model.to_json()
with open("model_512_WITH_10Layer_avgpool_kmean_80_epoch_.25_valid_warmup_3_batch_8.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_512_WITH_10Layer_avgpool_kmean_80_epoch_.25_valid_warmup_3_batch_8.h5")
print("Saved model to disk")

In [ ]:
# IMPLEMENTING sigmoid function

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

In [ ]:
# IMPLEMENTING softmax function

def softmax(x, axis=-1, t=-100.):
    x = x - np.max(x)
    
    if np.min(x) < t:
        x = x/np.min(x)*t
        
    e_x = np.exp(x)
    
    return e_x / e_x.sum(axis, keepdims=True)


In [ ]:
# Creating class object for each object

class BoundBox:
    def __init__(self, x, y, w, h, c = None, classes = None):
        self.x     = x
        self.y     = y
        self.w     = w
        self.h     = h
        
        self.c     = c
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):   # return class label
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self): # return score of object
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score

In [ ]:
# to normalize image on [0-1] scale
def Normalize(image):
    image = image / 255.
    
    return image



# calculating boxes IOU's
def bbox_iou(box1, box2):
    #print("box1 config = ", box1.x, box1.y, box1.w,box1.h)
    #print("box2 config = ", box2.x, box2.y, box2.w,box2.h)
    x1_min  = box1.x - box1.w/2
    x1_max  = box1.x + box1.w/2
    y1_min  = box1.y - box1.h/2
    y1_max  = box1.y + box1.h/2
    
    x2_min  = box2.x - box2.w/2
    x2_max  = box2.x + box2.w/2
    y2_min  = box2.y - box2.h/2
    y2_max  = box2.y + box2.h/2
    
    intersect_w = interval_overlap([x1_min, x1_max], [x2_min, x2_max])
    intersect_h = interval_overlap([y1_min, y1_max], [y2_min, y2_max])
    
    intersect = intersect_w * intersect_h
    
    union = box1.w * box1.h + box2.w * box2.h - intersect
    
    return float(intersect) / union



# calculating intersecting area     
def interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
            return 0
        else:
            return min(x2,x4) - x3  

     
        

In [ ]:
#draw boxes with their confidence value
import copy
def Draw_boxes(image, Boxes, labels):
   # print("boxes = ",Boxes)
    print(len(Boxes))
    #original_img = cv2.imread("foo.jpg")
    i=0
    Image = copy.copy(image)
    for box in Boxes:
        clone_img = copy.copy(image)
       # print("box cordinate = ",box.x,box.y,box.w,box.h)
        xmin  = int((box.x - box.w/2) * clone_img.shape[1])
        xmax  = int((box.x + box.w/2) * clone_img.shape[1])
        ymin  = int((box.y - box.h/2) * clone_img.shape[0])
        ymax  = int((box.y + box.h/2) * clone_img.shape[0])

        cv2.rectangle(clone_img, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
        cv2.rectangle(Image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
        cv2.putText(clone_img, 
                    labels[box.get_label()] + ' ' + str(box.get_score()), 
                    (xmin, ymin - 13), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1e-3 * clone_img.shape[0], 
                    (0,255,0), 2)
        
        cv2.putText(Image, 
                    labels[box.get_label()] + ' ' + str(box.get_score()), 
                    (xmin, ymin - 13), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1e-3 * Image.shape[0], 
                    (0,255,0), 2)
        cv2.imwrite("image"+str(i)+".jpg",clone_img)
        i=i+1
   # cv2.imwrite("image"+str(i)+".jpg",clone_img)
    return Image   

In [ ]:
#implement non max suppresion 

def Decode_netout(netout, obj_threshold, nms_threshold, anchors, nb_class):

    grid_h, grid_w, nb_box = netout.shape[:3]
  
    boxes = []
 
    # decode the output by the network
    netout[..., 4]  = sigmoid(netout[..., 4])

 
 
    netout[..., 5:] = netout[..., 4][..., np.newaxis] * softmax(netout[..., 5:])
    netout[..., 5:] *= netout[..., 5:] > obj_threshold

    for row in range(grid_h):
        for col in range(grid_w):
            for b in range(nb_box):
                # from 4th element onwards are confidence and class classes
                classes = netout[row,col,b,5:]
             
                if classes.any():
                    # first 4 elements are x, y, w, and h
                    x, y, w, h = netout[row,col,b,:4]
            
                    x = (col + sigmoid(x)) / grid_w              # center position, unit: image width
                    y = (row + sigmoid(y)) / grid_h              # center position, unit: image height
                    w = anchors[2 * b + 0] * np.exp(w) / grid_w  # unit: image width
                    h = anchors[2 * b + 1] * np.exp(h) / grid_h  # unit: image height
                    confidence = netout[row,col,b,4]
                 
                    
                    box = BoundBox(x, y, w, h, confidence, classes)
                    
                    boxes.append(box)
                    
 
    # suppress non-maximal boxes
    for c in range(nb_class):
        sorted_indices = list(reversed(np.argsort([box.classes[c] for box in boxes])))
       
 
        count = 0
        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]
           
            if boxes[index_i].classes[c] == 0: 
                continue
            else:
                for j in range(i+1, len(sorted_indices)):
                    index_j = sorted_indices[j]
                    
                    if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_threshold:
                        count=count + 1
                        boxes[index_j].classes[c] = 0
      
      
    # remove the boxes which are less likely than a obj_threshold
    boxes = [box for box in boxes if box.get_score() > obj_threshold]

  
    
    return boxes


In [ ]:
#loading model weights

os.chdir(r'/home/bitnami/vikrant/deep_version/saved_model')
from keras.models import model_from_json

# load json and create model
json_file = open('model_512_WITH_10Layer_avgpool_kmean_50_epoch_.25_valid_warmup_3_batch_8.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_512_WITH_10Layer_avgpool_kmean_50_epoch_.25_valid_warmup_3_batch_8.h5")
print("Loaded model from disk")

########################################

 #predicting 
dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))
test_image_index = 62

netout = loaded_model.predict([train_x[test_image_index:test_image_index+1,:,:, 0:1], dummy_array])
print("netout",netout.shape)
################################################
boxes = Decode_netout(netout[0], 
                      obj_threshold=OBJ_THRESHOLD,
                      nms_threshold=NMS_THRESHOLD,
                      anchors=ANCHORS, 
                      nb_class=CLASS)
print("boxes.shape = ",len(boxes))
#print(boxes)

#############################
os.chdir(r'/home/bitnami/vikrant/deep_version/output')

Image  = train_x[test_image_index]*255                                         # normalizing image back to [0-255] scale 
cv2.imwrite("orignal.jpg", Image)
############################

Image = Draw_boxes(Image, boxes, labels=LABELS)                                    #drawing boxes
#print("original_image_shape(train_x[67]) = ",type(original_image_size[67]))
image_true_height = original_image_size[67][0]
image_true_width = original_image_size[67][1]
#print("image_true_shape = ",image_true_height,image_true_width)

True_image = cv2.resize(Image, ( image_true_width,image_true_height))               # resizing image into original shape 
#print("True_image.shape = ",True_image.shape)
##########################################
os.chdir(r'/home/bitnami/vikrant/deep_version/output')
cv2.imwrite("predicted.jpg", True_image)
#print(Image.shape)

In [ ]:
OBJ_THRESHOLD    = 0.495          #.3                   # object threshold to remove lower confidence objects

NMS_THRESHOLD    = 0.3 